In [1]:
import prosperity3bt
import pandas as pd
import itertools

from prosperity3bt.runner import run_backtest
from prosperity3bt.models import TradeMatchingMode
from prosperity3bt.file_reader import FileSystemReader
from pathlib import Path

from importlib import reload

In [9]:
import ink_trader
reload(ink_trader)

def day_profit(result):
    last_timestamp = result.activity_logs[-1].timestamp

    profit = 0
    for row in reversed(result.activity_logs):
        if row.timestamp != last_timestamp:
            break

        profit += row.columns[-1]

    return profit

def backtest_days(trader, days=[-2, -1, 0, 1], data_path='.', round=[1,2]):
    profits = []
    for day in days:
        backtest_results = prosperity3bt.runner.run_backtest(trader, file_reader=FileSystemReader(Path(data_path)), round_num=round, day_num=day, print_output=False, trade_matching_mode=TradeMatchingMode.all, no_names=True, show_progress_bar=False)
        profit = day_profit(backtest_results)
        profits.append(profit)
    return profits

In [ ]:
def generate_param_combinations(param_grid):
    param_names = param_grid.keys()
    param_values = param_grid.values()
    combinations = list(itertools.product(*param_values))
    return [dict(zip(param_names, combination)) for combination in combinations]


param_grid = {
    'ink_change_threshold_pct': [0.005, 0.01, 0.02, 0.03, 0.05],
    # 'ink_change_threshold_pct': [0.005],
    'ink_window_size': [10,50,100,200,500],
    # 'weighted_mean': [False],
    'ink_position_limit': [50],
    'ignore_window_size': [10,20,40,50],
    'price_percent_needed': [0.01, 0.015, 0.02, 0.025]
    # 'price_percent_needed': [0.1],
}

param_profits = []

for params in generate_param_combinations(param_grid):
    trader = ink_trader.Trader(params)
    days = [-2, -1, 0]
    profits = backtest_days(trader, days=days, data_path='.', round=1)
    for d in days:
        params[f'profit_day_{d}'] = profits[days.index(d)]
    param_profits.append(params)

ValueError: Prices data is not available for round [1, 2] day -2

In [5]:
df = pd.DataFrame(param_profits)
df['tot_profit'] = df[[f'profit_day_{d}' for d in days]].sum(axis=1)
df.sort_values(by='tot_profit', ascending=False, inplace=True)
df.to_csv('backtests/LARGE_WINDOW_inktrader_bt_results.csv')

In [6]:
df

,ink_change_threshold_pct,ink_window_size,ink_position_limit,ignore_window_size,price_percent_needed,profit_day_-2,profit_day_-1,profit_day_0,tot_profit
0,0.005,10,50,10,0.10,5130.0,10531.00,273.0,15934.00
7,0.005,10,50,40,0.05,5130.0,10531.00,273.0,15934.00
1,0.005,10,50,10,0.05,5130.0,10531.00,273.0,15934.00
11,0.005,10,50,50,0.15,5130.0,10531.00,273.0,15934.00
10,0.005,10,50,50,0.05,5130.0,10531.00,273.0,15934.00
...,...,...,...,...,...,...,...,...,...
140,0.020,50,50,40,0.15,-11065.0,8415.00,2136.0,-514.00
138,0.020,50,50,40,0.10,-11065.0,8415.00,2136.0,-514.00
145,0.020,100,50,10,0.05,-9375.0,6556.75,2136.0,-682.25
146,0.020,100,50,10,0.15,-9375.0,4579.00,2136.0,-2660.00


In [7]:
# print any rows with all positive profits for days -2, -1, 0
positive_rows = df[(df['profit_day_-2'] > 0) & (df['profit_day_-1'] > 0) & (df['profit_day_0'] > 0)]
if not positive_rows.empty:
    print("Rows with all positive profits for days -2, -1, 0:")

positive_rows

Rows with all positive profits for days -2, -1, 0:


,ink_change_threshold_pct,ink_window_size,ink_position_limit,ignore_window_size,price_percent_needed,profit_day_-2,profit_day_-1,profit_day_0,tot_profit
0,0.005,10,50,10,0.10,5130.0,10531.00,273.0,15934.00
7,0.005,10,50,40,0.05,5130.0,10531.00,273.0,15934.00
1,0.005,10,50,10,0.05,5130.0,10531.00,273.0,15934.00
11,0.005,10,50,50,0.15,5130.0,10531.00,273.0,15934.00
10,0.005,10,50,50,0.05,5130.0,10531.00,273.0,15934.00
8,0.005,10,50,40,0.15,5130.0,10531.00,273.0,15934.00
9,0.005,10,50,50,0.10,5130.0,10531.00,273.0,15934.00
6,0.005,10,50,40,0.10,5130.0,10531.00,273.0,15934.00
5,0.005,10,50,20,0.15,5130.0,10531.00,273.0,15934.00
4,0.005,10,50,20,0.05,5130.0,10531.00,273.0,15934.00
